# 2.2- Data Preparation - Test.csv

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("datasets/TestDataAnalysis.csv")

In [3]:
df.head()

,App,Category,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,review_group
0,Bk Usha behn,LIFESTYLE,10,3.0M,"1,000+",Free,0,Everyone,Lifestyle,"January 14, 2018",1.0,4.1 and up,"[0, 23)"
1,Funny Ringtones,PERSONALIZATION,7146,14M,"1,000,000+",Free,0,Everyone 10+,Personalization,"February 2, 2018",1.1,4.0 and up,"[889, 26432)"
2,Strava.cz,SHOPPING,2221,Varies with device,"100,000+",Free,0,Everyone,Shopping,"July 10, 2018",Varies with device,Varies with device,"[889, 26432)"
3,BL Flowers Digital,SHOPPING,21,2.5M,100+,Free,0,Everyone,Shopping,"October 31, 2017",1.6,4.2 and up,"[0, 23)"
4,eCooltra: scooter sharing. Share electric scoo...,TRAVEL_AND_LOCAL,2822,27M,"100,000+",Free,0,Everyone,Travel & Local,"July 27, 2018",1.21.2,4.1 and up,"[889, 26432)"


## Feature Engineering content

* Delete `App` column, because is different in each row
* Change `Size` column values to millions or thousands
* Delete `Install` signature of + 
* Delete `$` symbol to Price
* Dummy encode `Categories, Content Rating, Genres`, because they don't have any range as ordinal data
* Last updated change to one column with year, one column with month, one column with day
* Delete `Current Ver`, because don't give any value
* In `Android Ver` edit number deleting the word "and up", and deleting the outliers with forma "-"
* Boolean values for `Type`
* Give an ordinal number to `review_group` as range of importance
* Delete `App, Size, Installs, Price, Last Updated, LastUpdatedNew, Current Ver, Android Ver, review_group` because is already encoded
* Impute Null values
* Normalization, Standarization???

### 1. Personalized Changes

#### 1.1 Changing Size format
From values with M and k, to floats

In [4]:
def change_M_k(df, new_column_name, column_name):
    df[new_column_name] = df[column_name].apply(lambda x: 
                         float(x.split("M")[0]) / 100 if x[-1] == "M" else 
                         float(x.split("k")[0]) / 100 if x[-1] == "k" else
                         np.nan
                    )
    return df

In [5]:
df = change_M_k(df, "SizeNew", "Size")

In [6]:
df.head()

,App,Category,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,review_group,SizeNew
0,Bk Usha behn,LIFESTYLE,10,3.0M,"1,000+",Free,0,Everyone,Lifestyle,"January 14, 2018",1.0,4.1 and up,"[0, 23)",0.030
1,Funny Ringtones,PERSONALIZATION,7146,14M,"1,000,000+",Free,0,Everyone 10+,Personalization,"February 2, 2018",1.1,4.0 and up,"[889, 26432)",0.140
2,Strava.cz,SHOPPING,2221,Varies with device,"100,000+",Free,0,Everyone,Shopping,"July 10, 2018",Varies with device,Varies with device,"[889, 26432)",NaN
3,BL Flowers Digital,SHOPPING,21,2.5M,100+,Free,0,Everyone,Shopping,"October 31, 2017",1.6,4.2 and up,"[0, 23)",0.025
4,eCooltra: scooter sharing. Share electric scoo...,TRAVEL_AND_LOCAL,2822,27M,"100,000+",Free,0,Everyone,Travel & Local,"July 27, 2018",1.21.2,4.1 and up,"[889, 26432)",0.270


#### 1.2 Changing installs "+" format for integers

In [7]:
def delimiter_format(df, new_column_name, column_name):
    df[new_column_name] = df[column_name].apply(lambda x: x.split("+")[0] if x[-1] == "+" else np.nan)
    df[new_column_name] = df[new_column_name].apply(lambda x: int(x.replace(',', '')))
    
    return df

In [8]:
df = delimiter_format(df, 'InstallsNew', 'Installs')

In [9]:
df.head()

,App,Category,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,review_group,SizeNew,InstallsNew
0,Bk Usha behn,LIFESTYLE,10,3.0M,"1,000+",Free,0,Everyone,Lifestyle,"January 14, 2018",1.0,4.1 and up,"[0, 23)",0.030,1000
1,Funny Ringtones,PERSONALIZATION,7146,14M,"1,000,000+",Free,0,Everyone 10+,Personalization,"February 2, 2018",1.1,4.0 and up,"[889, 26432)",0.140,1000000
2,Strava.cz,SHOPPING,2221,Varies with device,"100,000+",Free,0,Everyone,Shopping,"July 10, 2018",Varies with device,Varies with device,"[889, 26432)",NaN,100000
3,BL Flowers Digital,SHOPPING,21,2.5M,100+,Free,0,Everyone,Shopping,"October 31, 2017",1.6,4.2 and up,"[0, 23)",0.025,100
4,eCooltra: scooter sharing. Share electric scoo...,TRAVEL_AND_LOCAL,2822,27M,"100,000+",Free,0,Everyone,Travel & Local,"July 27, 2018",1.21.2,4.1 and up,"[889, 26432)",0.270,100000


#### 1.3 Changing symbol "$" in Pricing

In [10]:
def delete_price_symbol(df, new_column_name, column_name):
    df[new_column_name] = df[column_name].apply(lambda x: float(x.split("$")[1]) if x[0] == "$" else 0)
    return df

In [11]:
df = delete_price_symbol(df, "PriceNew", "Price")

In [12]:
df.head()

,App,Category,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,review_group,SizeNew,InstallsNew,PriceNew
0,Bk Usha behn,LIFESTYLE,10,3.0M,"1,000+",Free,0,Everyone,Lifestyle,"January 14, 2018",1.0,4.1 and up,"[0, 23)",0.030,1000,0.0
1,Funny Ringtones,PERSONALIZATION,7146,14M,"1,000,000+",Free,0,Everyone 10+,Personalization,"February 2, 2018",1.1,4.0 and up,"[889, 26432)",0.140,1000000,0.0
2,Strava.cz,SHOPPING,2221,Varies with device,"100,000+",Free,0,Everyone,Shopping,"July 10, 2018",Varies with device,Varies with device,"[889, 26432)",NaN,100000,0.0
3,BL Flowers Digital,SHOPPING,21,2.5M,100+,Free,0,Everyone,Shopping,"October 31, 2017",1.6,4.2 and up,"[0, 23)",0.025,100,0.0
4,eCooltra: scooter sharing. Share electric scoo...,TRAVEL_AND_LOCAL,2822,27M,"100,000+",Free,0,Everyone,Travel & Local,"July 27, 2018",1.21.2,4.1 and up,"[889, 26432)",0.270,100000,0.0


#### 1.4 Encoding Dates, separating each year, month and weekday

In [13]:
def change_date_format(df, new_column_name, column_name):
    df[new_column_name] = pd.to_datetime(df[column_name])
    df["{}Year".format(new_column_name)] = df[new_column_name].dt.year
    df["{}Month".format(new_column_name)] = df[new_column_name].dt.month
    df["{}Day".format(new_column_name)] = df[new_column_name].dt.day
    return df

In [14]:
df = change_date_format(df, "LastUpdatedNew", "Last Updated")

In [15]:
df.head()

,App,Category,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,review_group,SizeNew,InstallsNew,PriceNew,LastUpdatedNew,LastUpdatedNewYear,LastUpdatedNewMonth,LastUpdatedNewDay
0,Bk Usha behn,LIFESTYLE,10,3.0M,"1,000+",Free,0,Everyone,Lifestyle,"January 14, 2018",1.0,4.1 and up,"[0, 23)",0.030,1000,0.0,2018-01-14,2018,1,14
1,Funny Ringtones,PERSONALIZATION,7146,14M,"1,000,000+",Free,0,Everyone 10+,Personalization,"February 2, 2018",1.1,4.0 and up,"[889, 26432)",0.140,1000000,0.0,2018-02-02,2018,2,2
2,Strava.cz,SHOPPING,2221,Varies with device,"100,000+",Free,0,Everyone,Shopping,"July 10, 2018",Varies with device,Varies with device,"[889, 26432)",NaN,100000,0.0,2018-07-10,2018,7,10
3,BL Flowers Digital,SHOPPING,21,2.5M,100+,Free,0,Everyone,Shopping,"October 31, 2017",1.6,4.2 and up,"[0, 23)",0.025,100,0.0,2017-10-31,2017,10,31
4,eCooltra: scooter sharing. Share electric scoo...,TRAVEL_AND_LOCAL,2822,27M,"100,000+",Free,0,Everyone,Travel & Local,"July 27, 2018",1.21.2,4.1 and up,"[889, 26432)",0.270,100000,0.0,2018-07-27,2018,7,27


#### 1.5 Converting semversion to a float

In [16]:
def semversion_to_number(df, new_column_name, column_name):
    ## Find rare characters
    df[new_column_name] = df[column_name].apply(lambda x: 
                            x.split("W")[0] if x[-8] == "W" else
                            x.split("-")[0] if x[-7] == "-" else
                            x.split(" and up")[0] if x[-1] == "p" else
                            str(0)) ## change later this cero for NaN
    
    ## Find records with 3 poinbt version, like 4.0.2 and replace for float with first version
    df[new_column_name] = df[new_column_name].apply(lambda x: 
                            float(x.split(".")[0]) if x.count('.') == 2 else
                            float(x))
    
    ## Change zeros for NaNs
    df[new_column_name] = df[new_column_name].apply(lambda x: 
                            np.nan if x == 0 else
                            x)
    
    return df

In [17]:
df = semversion_to_number(df, "AndroidVerNew", "Android Ver")

In [18]:
df.head()

,App,Category,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,...,Android Ver,review_group,SizeNew,InstallsNew,PriceNew,LastUpdatedNew,LastUpdatedNewYear,LastUpdatedNewMonth,LastUpdatedNewDay,AndroidVerNew
0,Bk Usha behn,LIFESTYLE,10,3.0M,"1,000+",Free,0,Everyone,Lifestyle,"January 14, 2018",...,4.1 and up,"[0, 23)",0.030,1000,0.0,2018-01-14,2018,1,14,4.1
1,Funny Ringtones,PERSONALIZATION,7146,14M,"1,000,000+",Free,0,Everyone 10+,Personalization,"February 2, 2018",...,4.0 and up,"[889, 26432)",0.140,1000000,0.0,2018-02-02,2018,2,2,4.0
2,Strava.cz,SHOPPING,2221,Varies with device,"100,000+",Free,0,Everyone,Shopping,"July 10, 2018",...,Varies with device,"[889, 26432)",NaN,100000,0.0,2018-07-10,2018,7,10,NaN
3,BL Flowers Digital,SHOPPING,21,2.5M,100+,Free,0,Everyone,Shopping,"October 31, 2017",...,4.2 and up,"[0, 23)",0.025,100,0.0,2017-10-31,2017,10,31,4.2
4,eCooltra: scooter sharing. Share electric scoo...,TRAVEL_AND_LOCAL,2822,27M,"100,000+",Free,0,Everyone,Travel & Local,"July 27, 2018",...,4.1 and up,"[889, 26432)",0.270,100000,0.0,2018-07-27,2018,7,27,4.1


#### 1.6 Encoding review groups to oridinal numbers

In [19]:
def from_review_group_to_oridinal(df, new_column_name, column_name):
    df[new_column_name] = df[column_name].apply(
        lambda x: 0 if x == '[0, 23)' else 
        1 if x == '[23, 889)' else
        2 if x == '[889, 26432)' else
        3
    )
    return df

In [20]:
df = from_review_group_to_oridinal(df, "ReviewGroupNew", "review_group")

In [21]:
df.head()

,App,Category,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,...,review_group,SizeNew,InstallsNew,PriceNew,LastUpdatedNew,LastUpdatedNewYear,LastUpdatedNewMonth,LastUpdatedNewDay,AndroidVerNew,ReviewGroupNew
0,Bk Usha behn,LIFESTYLE,10,3.0M,"1,000+",Free,0,Everyone,Lifestyle,"January 14, 2018",...,"[0, 23)",0.030,1000,0.0,2018-01-14,2018,1,14,4.1,0
1,Funny Ringtones,PERSONALIZATION,7146,14M,"1,000,000+",Free,0,Everyone 10+,Personalization,"February 2, 2018",...,"[889, 26432)",0.140,1000000,0.0,2018-02-02,2018,2,2,4.0,2
2,Strava.cz,SHOPPING,2221,Varies with device,"100,000+",Free,0,Everyone,Shopping,"July 10, 2018",...,"[889, 26432)",NaN,100000,0.0,2018-07-10,2018,7,10,NaN,2
3,BL Flowers Digital,SHOPPING,21,2.5M,100+,Free,0,Everyone,Shopping,"October 31, 2017",...,"[0, 23)",0.025,100,0.0,2017-10-31,2017,10,31,4.2,0
4,eCooltra: scooter sharing. Share electric scoo...,TRAVEL_AND_LOCAL,2822,27M,"100,000+",Free,0,Everyone,Travel & Local,"July 27, 2018",...,"[889, 26432)",0.270,100000,0.0,2018-07-27,2018,7,27,4.1,2


### 2- Generic Changes

#### 2.1 Null values imputation

**Note:** Please note here that this data is for a competition, so I don't need to worry about data leakage

In [22]:
def null_exploration(df):
    ## to explore nulls
    print(df.isnull().sum())
    print(df.isnull().mean())

In [23]:
null_exploration(df)

App                      0
Category                 0
Reviews                  0
Size                     0
Installs                 0
Type                     0
Price                    0
Content Rating           0
Genres                   0
Last Updated             0
Current Ver              0
Android Ver              0
review_group             0
SizeNew                199
InstallsNew              0
PriceNew                 0
LastUpdatedNew           0
LastUpdatedNewYear       0
LastUpdatedNewMonth      0
LastUpdatedNewDay        0
AndroidVerNew          158
ReviewGroupNew           0
dtype: int64
App                    0.000000
Category               0.000000
Reviews                0.000000
Size                   0.000000
Installs               0.000000
Type                   0.000000
Price                  0.000000
Content Rating         0.000000
Genres                 0.000000
Last Updated           0.000000
Current Ver            0.000000
Android Ver            0.000000
review_gr

In [24]:
def null_imputation(column_name):
    df[column_name].fillna(df[column_name].mean(), inplace=True)
    return df

In [25]:
df = null_imputation("SizeNew")

In [26]:
df = null_imputation("AndroidVerNew")

#### 2.2 OneStepFeatEng

In [27]:
#### One step feature engineering

class OneStepFeatEng():
    def __init__(self, df):
        self.df = df
        
    # Drop columns
    def drop_column(self, column_names):
        self.df.drop(column_names, axis=1, inplace=True)
        return self.df.head()
    
    ## Separated cols
    def separated_cols(self, target_var):
        target_var = target_var
        features = [x for x in list(self.df.columns) if x != target_var]
        
        cat_cols = [column_name for column_name in self.df.columns if self.df[column_name].dtypes=='O']
        num_cols = [column_name for column_name in self.df.columns if self.df[column_name].dtypes!='O']
        
        return features, cat_cols, num_cols
    
    # Boolean encoding
    def boolean_encoding(self, column_name, true_val):
        self.df[column_name] = self.df[column_name].apply(lambda x: 1 if x == true_val else 0)
        return self.df.head()
    
    # Final columns
    def final_columns(self):
        return self.df.columns
    
    # Final dataframe
    def final_dataframe(self):
        return self.df

In [28]:
## Instance
X = OneStepFeatEng(df)

In [29]:
X.drop_column(['App', 'Size', 'Installs', 'Price', 'Last Updated', 'LastUpdatedNew', 'Current Ver', 
               'Android Ver', 'review_group'])

,Category,Reviews,Type,Content Rating,Genres,SizeNew,InstallsNew,PriceNew,LastUpdatedNewYear,LastUpdatedNewMonth,LastUpdatedNewDay,AndroidVerNew,ReviewGroupNew
0,LIFESTYLE,10,Free,Everyone,Lifestyle,0.030000,1000,0.0,2018,1,14,4.100000,0
1,PERSONALIZATION,7146,Free,Everyone 10+,Personalization,0.140000,1000000,0.0,2018,2,2,4.000000,2
2,SHOPPING,2221,Free,Everyone,Shopping,0.392887,100000,0.0,2018,7,10,3.837752,2
3,SHOPPING,21,Free,Everyone,Shopping,0.025000,100,0.0,2017,10,31,4.200000,0
4,TRAVEL_AND_LOCAL,2822,Free,Everyone,Travel & Local,0.270000,100000,0.0,2018,7,27,4.100000,2


In [30]:
X.boolean_encoding("Type", "Paid")

,Category,Reviews,Type,Content Rating,Genres,SizeNew,InstallsNew,PriceNew,LastUpdatedNewYear,LastUpdatedNewMonth,LastUpdatedNewDay,AndroidVerNew,ReviewGroupNew
0,LIFESTYLE,10,0,Everyone,Lifestyle,0.030000,1000,0.0,2018,1,14,4.100000,0
1,PERSONALIZATION,7146,0,Everyone 10+,Personalization,0.140000,1000000,0.0,2018,2,2,4.000000,2
2,SHOPPING,2221,0,Everyone,Shopping,0.392887,100000,0.0,2018,7,10,3.837752,2
3,SHOPPING,21,0,Everyone,Shopping,0.025000,100,0.0,2017,10,31,4.200000,0
4,TRAVEL_AND_LOCAL,2822,0,Everyone,Travel & Local,0.270000,100000,0.0,2018,7,27,4.100000,2


In [31]:
X.final_dataframe().Type.value_counts()

0    1339
1     109
Name: Type, dtype: int64

In [32]:
features, cat_cols, num_cols = X.separated_cols('Rating')

In [33]:
X.final_dataframe().head()

,Category,Reviews,Type,Content Rating,Genres,SizeNew,InstallsNew,PriceNew,LastUpdatedNewYear,LastUpdatedNewMonth,LastUpdatedNewDay,AndroidVerNew,ReviewGroupNew
0,LIFESTYLE,10,0,Everyone,Lifestyle,0.030000,1000,0.0,2018,1,14,4.100000,0
1,PERSONALIZATION,7146,0,Everyone 10+,Personalization,0.140000,1000000,0.0,2018,2,2,4.000000,2
2,SHOPPING,2221,0,Everyone,Shopping,0.392887,100000,0.0,2018,7,10,3.837752,2
3,SHOPPING,21,0,Everyone,Shopping,0.025000,100,0.0,2017,10,31,4.200000,0
4,TRAVEL_AND_LOCAL,2822,0,Everyone,Travel & Local,0.270000,100000,0.0,2018,7,27,4.100000,2


In [34]:
features

['Category',
 'Reviews',
 'Type',
 'Content Rating',
 'Genres',
 'SizeNew',
 'InstallsNew',
 'PriceNew',
 'LastUpdatedNewYear',
 'LastUpdatedNewMonth',
 'LastUpdatedNewDay',
 'AndroidVerNew',
 'ReviewGroupNew']

In [35]:
cat_cols

['Category', 'Content Rating', 'Genres']

In [36]:
num_cols

['Reviews',
 'Type',
 'SizeNew',
 'InstallsNew',
 'PriceNew',
 'LastUpdatedNewYear',
 'LastUpdatedNewMonth',
 'LastUpdatedNewDay',
 'AndroidVerNew',
 'ReviewGroupNew']

In [37]:
X.final_dataframe().to_csv("datasets/TestFeatureEngined.csv", index=False)